In [37]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu118
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^
Note: you may need to restart the kernel to use updated packages.


In [38]:
!pip install transformers requests beautifulsoup4 pandas numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
    sys-platform (=="darwin") ; extra == 'objc'
                 ~^


In [41]:
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import re
from bs4 import BeautifulSoup
import requests

In [8]:
import os
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm import tqdm

# Disable notebook mode
tqdm.notebook = False


In [9]:
# manually downloaded the model in models
model_path = os.path.expanduser("~/models/nlptown/bert-base-multilingual-uncased-sentiment")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=True)

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(model_path)


In [31]:
tokens = tokenizer.encode('I was good but couldve been better', return_tensors='pt')

In [32]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-1.7736,  0.1815,  2.0361,  1.1457, -1.3631]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [33]:
tokenizer.decode(tokens[0])


'[CLS] i was good but couldve been better [SEP]'

In [34]:
result.logits

tensor([[-1.7736,  0.1815,  2.0361,  1.1457, -1.3631]],
       grad_fn=<AddmmBackward0>)

In [35]:
torch.argmax(result.logits) + 1

tensor(3)

In [ ]:
## Collect Reviews


In [66]:
r = requests.get('https://www.zomato.com/bangalore/meghana-foods-residency-road/reviews')
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('.*sc-1hez2tp-0 sc-hnEmYr gPrxrQ.*')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [68]:
soup

<html><head>
<title>Access Denied</title>
</head><body>
<h1>Access Denied</h1>
 
You don't have permission to access "http://www.zomato.com/bangalore/meghana-foods-residency-road/reviews" on this server.<p>
Reference #18.25d70b17.1722854821.248a5c
<p>https://errors.edgesuite.net/18.25d70b17.1722854821.248a5c</p>
</p></body>
</html>

In [128]:

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Fetch the webpage
r = requests.get('https://www.zomato.com/bangalore/dominos-pizza-1-mg-road/reviews', headers=headers)

# Parse the webpage content
soup = BeautifulSoup(r.text, 'html.parser')

# Regular expression pattern to match the specific class pattern
pattern = re.compile(r'sc-1hez2tp-0 sc-.*')

# Custom function to check if the tag's class attribute matches the pattern
def matches_class_pattern(tag):
    if tag.name == 'p' and tag.has_attr('class'):
        class_attr = ' '.join(tag['class'])
        return pattern.search(class_attr) is not None
    return False

# Find all <p> tags that match the class pattern
results = soup.find_all(matches_class_pattern)

# Extract the text from each matched element
reviews = [result.text for result in results]

# Print the reviews
for review in reviews:
    print(review)


Detect current location
Using GPS
View Gallery
Samson
Weirdest experience of ordering from dominos but the pizza was great. The delivery not so much. The delivery guy mistaked my order for someone else's but then again even I got confused. I do appreciate the fact that he came back to deliver the right order but man I wish he never got it wrong in the first place. I get that the order rush he had was much but I wish he organised himself well. Thanks you anyway for the great food, the food was delicious but my delivery experience not so much.
815287
they are never on time, all promises are fake
Tresa
food is cool 
Ayush Kumar
no mushroom was added
Vimal
they haven't delivered the ordered food , just scamming the customer even that delivery guy just is a fraud I will also raise this issue now 


In [129]:
reviews

['Detect current location',
 'Using GPS',
 'View Gallery',
 'Samson',
 "Weirdest experience of ordering from dominos but the pizza was great. The delivery not so much. The delivery guy mistaked my order for someone else's but then again even I got confused. I do appreciate the fact that he came back to deliver the right order but man I wish he never got it wrong in the first place. I get that the order rush he had was much but I wish he organised himself well. Thanks you anyway for the great food, the food was delicious but my delivery experience not so much.",
 '815287',
 'they are never on time, all promises are fake',
 'Tresa',
 'food is cool ',
 'Ayush Kumar',
 'no mushroom was added',
 'Vimal',
 "they haven't delivered the ordered food , just scamming the customer even that delivery guy just is a fraud I will also raise this issue now "]

In [130]:
reviews = reviews[3:]

In [131]:
reviews

['Samson',
 "Weirdest experience of ordering from dominos but the pizza was great. The delivery not so much. The delivery guy mistaked my order for someone else's but then again even I got confused. I do appreciate the fact that he came back to deliver the right order but man I wish he never got it wrong in the first place. I get that the order rush he had was much but I wish he organised himself well. Thanks you anyway for the great food, the food was delicious but my delivery experience not so much.",
 '815287',
 'they are never on time, all promises are fake',
 'Tresa',
 'food is cool ',
 'Ayush Kumar',
 'no mushroom was added',
 'Vimal',
 "they haven't delivered the ordered food , just scamming the customer even that delivery guy just is a fraud I will also raise this issue now "]

In [132]:
# temp = []
# for i in range(0, len(reviews)):
#     if i % 2 != 0:
#         temp.append(reviews[i])

# reviews = temp
# reviews
        
    
# Get every second element starting from index 1
reviews = reviews[1::2]


In [133]:
reviews

["Weirdest experience of ordering from dominos but the pizza was great. The delivery not so much. The delivery guy mistaked my order for someone else's but then again even I got confused. I do appreciate the fact that he came back to deliver the right order but man I wish he never got it wrong in the first place. I get that the order rush he had was much but I wish he organised himself well. Thanks you anyway for the great food, the food was delicious but my delivery experience not so much.",
 'they are never on time, all promises are fake',
 'food is cool ',
 'no mushroom was added',
 "they haven't delivered the ordered food , just scamming the customer even that delivery guy just is a fraud I will also raise this issue now "]

In [ ]:
## Load reviews into dataframe


In [134]:
df = pd.DataFrame(np.array(reviews), columns=['review'])

In [135]:
df

,review
0,Weirdest experience of ordering from dominos b...
1,"they are never on time, all promises are fake"
2,food is cool
3,no mushroom was added
4,"they haven't delivered the ordered food , just..."


In [136]:
df['review'].iloc[1]

'they are never on time, all promises are fake'

In [137]:
## Sentiment Function
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [138]:
sentiment_score(df['review'].iloc[1])

1

In [139]:
df['sentiment'] = df['review'].apply(lambda x: sentiment_score(x))


In [140]:
df

,review,sentiment
0,Weirdest experience of ordering from dominos b...,3
1,"they are never on time, all promises are fake",1
2,food is cool,4
3,no mushroom was added,1
4,"they haven't delivered the ordered food , just...",1
